In [2]:
import geopandas as gpd
import pandas as pd
import structlog
import yaml
from shapely import from_wkt


def create_lookup(essential_amenities: dict[str, list[str]]) -> dict[str, str]:
    result = {}
    for cat, osm_list in essential_amenities.items():
        for i in osm_list:
            result[i] = cat
    return result


with open("../data/essential_amenities.yaml") as fp:
    essential_amenities = yaml.safe_load(fp)

logger = structlog.get_logger()

lookup = create_lookup(essential_amenities)

In [3]:
CITY = "budapest"
CITY = "paris"

In [4]:
amenities = gpd.read_file(
    f"../output/{CITY}/amenities/amenities_filtered.wkt.csv", engine="pyogrio"
)
amenities.rename({"category": "osm_category"}, axis=1, inplace=True)
amenities["category"] = amenities["osm_category"].map(lookup)
amenities.dropna(subset=["category"], inplace=True)
amenities.drop(["osm_type", "amenity_type", "amenity_subtype"], axis=1, inplace=True)
amenities["geometry"] = amenities["geometry"].apply(from_wkt)
amenities = gpd.GeoDataFrame(amenities, geometry="geometry", crs=4326)
amenities.head(3)

,osm_id,osm_category,name,geometry_type,geometry,category
89,27091941,amenity:cinema,Le Capitole,Point,POINT (2.22871 48.87055),cultural_institutions
91,27091971,amenity:post_office,La Poste Suresnes Hôtel-de-Ville,Point,POINT (2.22557 48.87002),services
94,27415802,amenity:post_office,Paris 17 Wagram,Point,POINT (2.30221 48.88328),services


In [5]:
isochrones = gpd.read_file(f"../output/{CITY}/isochrones.geojson", engine="pyogrio")
isochrones.head(3)

,stop_id,costing,range,geometry
0,9375373,walk,5,"POLYGON ((2.55953 49.01458, 2.55881 49.01322, ..."
1,9375373,walk,10,"POLYGON ((2.55653 49.01653, 2.55572 49.01431, ..."
2,9375373,walk,15,"POLYGON ((2.55553 49.01995, 2.55453 49.01895, ..."


In [6]:
result = pd.DataFrame()
for i in isochrones[["costing", "range"]].drop_duplicates().itertuples():
    ac = (
        isochrones.query(f"range == {i.range} & costing=='{i.costing}'")
        .sjoin(amenities)
        .groupby(["stop_id", "category"])["osm_id"]
        .count()
        .reset_index()
    )
    ac.rename({"osm_id": "count"}, axis=1, inplace=True)

    ac = (
        pd.pivot_table(ac, index=["stop_id"], columns=["category"], values="count")
        .fillna(0)
        .map(int)
        .reset_index()
    )

    ac["costing"] = i.costing
    ac["range"] = i.range
    result = pd.concat([result, ac])

result.to_csv(f"../output/{CITY}/amenity_counts_in_accessibility.csv", index=False)

In [7]:
sgfw = gpd.read_file(f"../output/{CITY}/stop_geometries_from_walk.geojson")

In [8]:
sgfw.head(3)

,stop_id,area,ellipticity,geometry
0,9375373,0.306,NaN,"POLYGON ((2.55953 49.01458, 2.56197 49.0135, 2..."
1,9375374,0.306,NaN,"POLYGON ((2.55953 49.01458, 2.56197 49.0135, 2..."
2,3672258,0.871,0.7305,"MULTIPOLYGON (((2.56836 49.00184, 2.56833 49.0..."


In [9]:
temp = sgfw.sjoin(amenities)
temp = temp.groupby(["stop_id", "category"])["osm_id"].count().reset_index()
temp.rename({"osm_id": "count"}, axis=1, inplace=True)
temp = (
    pd.pivot_table(temp, index=["stop_id"], columns=["category"], values="count")
    .fillna(0)
    .map(int)
    .reset_index()
)
temp["costing"] = "public_transport"
# temp["range"] = "10+5"
temp["range"] = pd.NA
temp.to_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv",
    index=False,
)